In [1]:
import os
from cot import Collection
from cot.generate import FRAGMENTS
from rich.pretty import pprint
import json
from sprint_utils import get_answer_extraction, join_strings

In [2]:
# "commonsense_qa", "validation"
# "med_qa", "test"
# "medmc_qa", "validation"
# "open_book_qa", "test"
# "strategy_qa", "train"
# "worldtree", "test"

In [3]:
# None,         # no cot-trigger
# 'kojima-01',  # Answer: Let's think step by step.
# 'kojima-02',  # Answer: First,
# 'kojima-03',  # Answer: Let's think about this logically.

In [4]:
def model_sprint_thoughtsource(collection, dataset_name_splits, api_service_model, cot_trigger_key=None):
    # collection = create_thoughtsource_100()
    collection = collection
    for dataset_name, split in dataset_name_splits:
        print(dataset_name)
        answer_extraction_key = get_answer_extraction(collection[dataset_name][split][0]["type"], collection[dataset_name][split][0]["choices"])
        for api_service, model, api_time_interval in api_service_model:

            config={
                "instruction_keys": None,
                "cot_trigger_keys": cot_trigger_key,
                "answer_extraction_keys": [answer_extraction_key], # Therefore, among A through C/D/E/F, the answer is'
                "author" : "thoughtsource",
                "api_service": api_service,
                "engine": model,
                "temperature": 0,
                "max_tokens": 512,
                "api_time_interval": api_time_interval,
                "verbose": False,
                "warn": False,
            }
        
            collection.generate(name=dataset_name, split=split,config=config)
            collection.dump("extra_save_thoughtsource_100_" + dataset_name + "_huggingface_endpoint.json")
    collection.evaluate()
    # write model name manually in here, or it gives an error because of the "/" in the model name = endpoint url in this case
    collection.dump("thoughtsource_100" + "_" + api_service + "_" + "google_flan-t5-xxl" + join_strings(config["cot_trigger_keys"]) + ".json")

In [5]:
# ENDPOINT TEST
endpoint_url = (
    "https://fcyrfns6if4vl115.us-east-1.aws.endpoints.huggingface.cloud"
)

In [7]:
dataset_name_splits = [
    # ("commonsense_qa", "validation"),
    # ("med_qa", "test"),
    # ("medmc_qa", "validation"),
    ("open_book_qa", "test"),
    ("strategy_qa", "train"),
    ("worldtree", "test"),
]

# api_service_model = [("huggingface_hub", "google/flan-t5-xl", 1)]
api_service_model = [("huggingface_endpoint", endpoint_url, 0)]
# api_service_model = [("mock_api", "mock_model", 0)]
cot_trigger_key = [None, "kojima-01"]

In [7]:
# from cot.create import create_thoughtsource_1
# coll = create_thoughtsource_1()
# coll.dump("thoughtsource_data/thoughtsource_1_empty.json")

In [9]:
# collection = Collection.from_json("thoughtsource_data/thoughtsource_1_empty.json")
collection = Collection.from_json("thoughtsource_data/thoughtsource_100_empty.json")

[nltk_data] Downloading package punkt to /home/kon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
model_sprint_thoughtsource(collection, dataset_name_splits, api_service_model, cot_trigger_key)

open_book_qa
Generating open_book_qa...


  0%|          | 0/100 [00:00<?, ?ex/s]

API-Error in item 36: Error raised by inference endpoint: ('Connection aborted.', ConnectionResetError(104, 'Connection reset by peer'))
Retrying with additional time of 10 seconds.


Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Error raised by inference endpoint: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [11]:
eval = collection.evaluate()
pprint(eval)

Evaluating commonsense_qa...


  0%|          | 0/100 [00:00<?, ?ex/s]

Evaluating med_qa...


  0%|          | 0/100 [00:00<?, ?ex/s]

Evaluating medmc_qa...


...

Evaluating strategy_qa...


  0%|          | 0/100 [00:00<?, ?ex/s]

Evaluating worldtree...


  0%|          | 0/100 [00:00<?, ?ex/s]

{
│   'commonsense_qa': {'validation': {}},
│   'med_qa': {'test': {}},
│   'medmc_qa': {'validation': {}},
│   'open_book_qa': {
│   │   'test': {
│   │   │   'accuracy': {
│   │   │   │   'https://fcyrfns6if4vl115.us-east-1.aws.endpoints.huggingface.cloud': {
│   │   │   │   │   'None_None_kojima-A-D': 0.82,
│   │   │   │   │   'None_kojima-01_kojima-A-D': 0.79
│   │   │   │   }
│   │   │   }
│   │   }
│   },
│   'strategy_qa': {
│   │   'train': {
│   │   │   'accuracy': {
│   │   │   │   'https://fcyrfns6if4vl115.us-east-1.aws.endpoints.huggingface.cloud': {
│   │   │   │   │   'None_None_kojima-yes-no': 0.61,
│   │   │   │   │   'None_kojima-01_kojima-yes-no': 0.69
│   │   │   │   }
│   │   │   }
│   │   }
│   },
│   'worldtree': {
│   │   'test': {
│   │   │   'accuracy': {
│   │   │   │   'https://fcyrfns6if4vl115.us-east-1.aws.endpoints.huggingface.cloud': {
│   │   │   │   │   'None_None_kojima-A-D': 0.851485,
│   │   │   │   │   'None_kojima-01_kojima-A-D': 0.8
│   │   │   │   }
│   │   │   }
│   │   }
│   }
}